In [ ]:
import keras
import os
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.model_selection import KFold

In [ ]:
final_test = True #@param {type: "boolean"}

## Cora

In [ ]:
zip_file = keras.utils.get_file(
    fname="cora.tgz",
    origin="https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz",
    extract=True,
)
data_dir = os.path.join(os.path.dirname(zip_file), "cora")
data_name = "cora.cites"
node_data_name = 'cora.content'

In [ ]:
os.path.dirname(zip_file)

'/root/.keras/datasets'

## Pubmed

In [ ]:
zip_file = keras.utils.get_file(
    fname="pubmed.tgz",
    origin="https://linqs-data.soe.ucsc.edu/public/Pubmed-Diabetes.tgz",
    extract=True,
)
data_name = "data/Pubmed-Diabetes.DIRECTED.cites.tab"
node_data_name = "data/Pubmed-Diabetes.NODE.paper.tab"
data_dir = os.path.join(os.path.dirname(zip_file), "Pubmed-Diabetes")


14710584/14710584 [==============================] - 2s 0us/step


In [ ]:
os.listdir(data_dir)

['README', 'citeseer.cites', 'citeseer.content']

# CiteSeer

In [ ]:
zip_file = keras.utils.get_file(
    fname="citeseer.tgz",
    origin="https://linqs-data.soe.ucsc.edu/public/datasets/citeseer-doc-classification/citeseer-doc-classification.zip",
    extract=True,
)
data_name = "citeseer.cites"
node_data_name = "citeseer.content"
data_dir = os.path.join(os.path.dirname(zip_file), "citeseer-doc-classification")

356438/356438 [==============================] - 0s 1us/step


# OGB Mag

In [ ]:
%pip install ogb torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 55.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=9f00d9e02523ee410695378b78f816720e56e4fc363806f02ba03fce5fa57366
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=83ff91959dec33d5529b9c57549b0b102484d8ac4f7a669cc5c9ae86ee6f5192
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built tor

In [ ]:
import ogb
from ogb.nodeproppred import NodePropPredDataset 
from torch_geometric.data import DataLoader
#H:\\PhD Research\Data\PROTEINS\PROTEINS_node_attributes.txt
# Download and process data at './dataset/ogbg_molhiv/'
dataset = NodePropPredDataset(name = "ogbn-mag", root = '/Users/joshem/PhD Research/Data/')
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0] # pyg graph object

Downloaded 0.40 GB: 100%|██████████| 413/413 [00:29<00:00, 13.88it/s]


Extracting /Users/joshem/PhD Research/Data/mag.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 8439.24it/s]

Saving...


In [ ]:
import requests, zipfile, io
r = requests.get('https://utdallas.box.com/shared/static/19yvbjlxzkwhjft5k9r9sfu23pkvm4aa.zip', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('/content')

In [ ]:
data = pd.read_csv('/content/Feature_nbd_mag.csv', index_col=0)

In [ ]:
len(data)

736389

In [ ]:
import numpy as np

p_data = open(os.path.join(data_dir, f'{data_name}'))


# Create a list from the data
myls=list(p_data)

# Remove the new line symbole from the list
mylist = [line.rstrip('\n') for line in myls]
if data_name == 'data/Pubmed-Diabetes.DIRECTED.cites.tab':
  mylist = [line.split('\t')[1][6:] + '\t' + line.split('\t')[3][6:] 
              for line in mylist[2:]]
#print(mylist)
#print(graph_ind)
#print(graph_level)

p_data.close()

In [ ]:
mylist

['19127292\t17363749',
 '19668377\t17293876',
 '1313726\t3002783',
 '19110882\t14578298',
 '18606979\t10333910',
 '19735543\t8420806',
 '19228402\t17017752',
 '19220880\t12734781',
 '16595006\t11790819',
 '19143817\t11206408',
 '11943852\t7722468',
 '18644886\t3138126',
 '8325993\t2186054',
 '18509207\t15043684',
 '17894830\t2044434',
 '9326333\t7487991',
 '19228405\t11561483',
 '17144912\t1324940',
 '1541672\t1991568',
 '18349042\t12486503',
 '9737664\t7594559',
 '17174749\t1361076',
 '8100835\t2180755',
 '11791216\t11484155',
 '15192149\t11869680',
 '16127462\t9734395',
 '17327460\t9758619',
 '8529129\t1936620',
 '3280182\t6884606',
 '14585101\t9732339',
 '18477407\t15968442',
 '16846517\t12171451',
 '12560454\t7024027',
 '12975475\t11907129',
 '16075046\t1284550',
 '18436707\t15384883',
 '8145050\t1972779',
 '11522610\t2444321',
 '11454665\t2933287',
 '18406405\t9032105',
 '7833731\t2951066',
 '11091269\t9317167',
 '19360314\t12829651',
 '7567975\t3240835',
 '3891786\t6311651',
 '16

In [ ]:
import os
import pandas as pd
if node_data_name == "data/Pubmed-Diabetes.NODE.paper.tab":
  node_data = []
  with open(os.path.join(data_dir, node_data_name)) as f:
    for line in f.readlines()[2:]:
      node_info = {d.split('=')[0]:d.split('=')[1] for d in line.split('\t')[1:]}
      node_info['ID'] = line.split('\t')[0]
      node_info['class'] = int(node_info.pop('label'))-1
      node_data.append(node_info)
    # print(node_data[1])
  node_data = pd.DataFrame(node_data).fillna(0.0).drop('summary', axis=1)
else:
  num_features = 0
  with open(os.path.join(data_dir, node_data_name)) as f:
    num_features = len(f.readline().split())
  feature_names = ["w_{}".format(ii) for ii in range(num_features-2)]
  column_names = ["ID"]+ feature_names + ["class"]
  node_data = pd.read_csv(os.path.join(data_dir, node_data_name), sep='\t', header=None, names=column_names,
                          dtype={'ID': str})

  
node_data.head(10)


,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
5,1126012,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Probabilistic_Methods
6,1107140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Theory
7,1102850,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neural_Networks
8,31349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Neural_Networks
9,1106418,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Theory


In [ ]:
node_data_name

'data/Pubmed-Diabetes.NODE.paper.tab'

In [ ]:
if node_data_name == 'cora.content':
  Data=node_data.replace({'class': {'Case_Based':0 , 'Genetic_Algorithms': 1,'Neural_Networks':2,'Probabilistic_Methods':3 , 'Reinforcement_Learning': 4,'Rule_Learning':5,'Theory':6}})
  # Case_Based==0
  #		Genetic_Algorithms==1
  #		Neural_Networks==2
  #		Probabilistic_Methods==3
  #		Reinforcement_Learning==4
  #		Rule_Learning==5
  #		Theory==6
  Data.head(10)

elif node_data_name == 'citeseer.content':
  Data=node_data.replace({'class': {'Agents':0 , 'AI': 1,'DB':2,'IR':3 , 'ML': 4,'HCI':5}})
  Data.head(10)
else:
  Data = node_data

In [ ]:
Data.head(10)

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,class
0,100157,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,105684,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,114091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,11510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,115971,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,117999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
8,120432,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,126894,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [ ]:
data_s=Data.sort_values(by=['ID'],ascending=True)
data_s.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,class
1622,1000012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2044,100197,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
32,100701,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,100935,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1119,100961,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
print(list(data_s.columns))

['w-rat', 'w-common', 'w-use', 'w-examin', 'w-pathogenesi', 'w-retinopathi', 'w-mous', 'w-studi', 'w-anim', 'w-model', 'w-metabol', 'w-abnorm', 'w-contribut', 'w-develop', 'w-investig', 'w-mice', 'w-2', 'w-month', 'w-compar', 'w-obtain', 'w-method', 'w-induc', 'w-6', 'w-inject', 'w-experiment', 'w-normal', 'w-diet', 'w-30', 'w-hyperglycemia', 'w-level', 'w-lipid', 'w-oxid', 'w-activ', 'w-protein', 'w-kinas', 'w-c', 'w-measur', 'w-result', 'w-increas', 'w-retin', 'w-stress', 'w-3', 'w-similar', 'w-observ', 'w-conclus', 'w-play', 'w-import', 'w-role', 'w-present', 'summary', 'ID', 'class', 'w-p', 'w-m', 'w-r', 'w-muscl', 'w-control', 'w-chang', 'w-dure', 'w-lower', 'w-higher', 'w-mass', 'w-correl', 'w-decreas', 'w-determin', 'w-concentr', 'w-stimul', 'w-period', 'w-caus', 'w-mark', 'w-group', 'w-evid', 'w-fast', 'w-type', 'w-signific', 'w-differ', 'w-ratio', 'w-suggest', 'w-degre', 'w-occur', 'w-vivo', 'w-respect', 'w-dysfunct', 'w-region', 'w-high', 'w-appear', 'w-sever', 'w-affect', 'w

In [ ]:
mynode=list(data_s['ID'])

#print(mynode)
node_ID1=np.unique(mynode)
node_ID= {node: i for i, node in enumerate(node_ID1)}
print(node_ID)
n=len(node_ID)
print(n)

{'1000012': 0, '100197': 1, '100701': 2, '100935': 3, '100961': 4, '101143': 5, '101145': 6, '101261': 7, '101263': 8, '101660': 9, '101662': 10, '10169': 11, '10174': 12, '10177': 13, '101811': 14, '10183': 15, '10186': 16, '102061': 17, '1022969': 18, '102406': 19, '1026': 20, '102879': 21, '102884': 22, '102938': 23, '102939': 24, '1031453': 25, '1033': 26, '1034': 27, '103430': 28, '103482': 29, '1035': 30, '103515': 31, '103528': 32, '103529': 33, '103531': 34, '103537': 35, '103543': 36, '10430': 37, '10435': 38, '104840': 39, '105057': 40, '1050679': 41, '10531': 42, '105856': 43, '105865': 44, '105899': 45, '1059953': 46, '1061127': 47, '1063773': 48, '106590': 49, '107177': 50, '1071981': 51, '107251': 52, '107252': 53, '107569': 54, '10793': 55, '10796': 56, '10798': 57, '108047': 58, '108962': 59, '108963': 60, '108974': 61, '108983': 62, '109323': 63, '1095507': 64, '10981': 65, '110041': 66, '110162': 67, '110163': 68, '110164': 69, '1102364': 70, '1102400': 71, '1102407':

In [ ]:
len(set(node_ID.keys()))

19717

In [ ]:
A=np.zeros((n,n), dtype=int)
i=0
for d in mylist:
    d=mylist[i].split("\t")
    
    if d[0] in node_ID and d[1] in node_ID:
      
      A[node_ID[d[0]]][node_ID[d[1]]] = 1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
data_s

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,class
0,100157,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2177,101570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1011,10227,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2178,102637,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,zhu00incorporating,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2175,zhu00segmenting,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2176,zhu98line,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1009,zini01caselp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
with open(os.path.join(data_dir, node_data_name)) as f:
  for line in f.readlines()[:50]:
    print(line)


NODE	paper

cat=1,2,3:label	numeric:w-rat:0.0	numeric:w-common:0.0	numeric:w-use:0.0	numeric:w-examin:0.0	numeric:w-pathogenesi:0.0	numeric:w-retinopathi:0.0	numeric:w-mous:0.0	numeric:w-studi:0.0	numeric:w-anim:0.0	numeric:w-model:0.0	numeric:w-metabol:0.0	numeric:w-abnorm:0.0	numeric:w-contribut:0.0	numeric:w-develop:0.0	numeric:w-investig:0.0	numeric:w-mice:0.0	numeric:w-2:0.0	numeric:w-month:0.0	numeric:w-compar:0.0	numeric:w-obtain:0.0	numeric:w-method:0.0	numeric:w-induc:0.0	numeric:w-6:0.0	numeric:w-inject:0.0	numeric:w-experiment:0.0	numeric:w-normal:0.0	numeric:w-diet:0.0	numeric:w-30:0.0	numeric:w-hyperglycemia:0.0	numeric:w-level:0.0	numeric:w-lipid:0.0	numeric:w-oxid:0.0	numeric:w-activ:0.0	numeric:w-protein:0.0	numeric:w-kinas:0.0	numeric:w-c:0.0	numeric:w-measur:0.0	numeric:w-result:0.0	numeric:w-increas:0.0	numeric:w-retin:0.0	numeric:w-stress:0.0	numeric:w-3:0.0	numeric:w-similar:0.0	numeric:w-observ:0.0	numeric:w-conclus:0.0	numeric:w-play:0.0	numeric:w-import:0.0	nume

In [ ]:
set(node_ID.keys()) - set(data_s['ID'])

set()

In [ ]:
catagories=data_s['class'].to_numpy()
print(catagories)

[5 2 0 ... 2 2 2]


In [ ]:
max(catagories)

6

In [ ]:
import random
train_splits = []
test_splits = []
for _ in range(10):
  p=random.sample(range(len(catagories)), 1000)
  #print(p)
  q=[]
  for i in range(n):
      if i not in p:
          q.append(i)
  print(q)
  train_splits.append(q)
  test_splits.append(p)

[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 19, 24, 25, 26, 27, 28, 31, 32, 33, 37, 38, 39, 41, 43, 45, 46, 47, 48, 49, 50, 51, 55, 56, 62, 63, 64, 65, 66, 67, 68, 70, 71, 73, 74, 75, 77, 78, 80, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 96, 97, 100, 101, 103, 104, 109, 110, 111, 112, 113, 117, 118, 119, 122, 125, 127, 128, 129, 131, 133, 134, 135, 138, 139, 141, 143, 146, 147, 148, 149, 153, 154, 155, 156, 157, 158, 161, 164, 165, 166, 167, 169, 175, 177, 180, 181, 182, 186, 188, 189, 191, 193, 194, 195, 196, 197, 200, 201, 203, 204, 207, 209, 211, 213, 215, 216, 218, 221, 222, 226, 229, 231, 232, 236, 237, 240, 241, 242, 244, 245, 246, 247, 250, 252, 254, 255, 256, 259, 260, 261, 262, 264, 265, 266, 267, 268, 269, 271, 273, 274, 275, 276, 277, 279, 280, 283, 284, 288, 289, 291, 292, 293, 295, 297, 298, 299, 300, 302, 303, 304, 305, 306, 308, 309, 310, 312, 313, 314, 315, 316, 318, 320, 322, 323, 324, 326, 329, 330, 331, 332, 333, 334, 335, 336, 339, 340, 341, 342, 343, 

In [ ]:
class_60 = []
if final_test:
  class_60=[np.array(catagories) for _ in range(10)]
  for i in range(10):
    for d in test_splits[i]:
      class_60[i][d]= max(catagories) + 1
else:
  class_60 = np.array(catagories)
  for d in test_splits[0]:
    class_60[d] = max(catagories) + 1
print(class_60)

[array([5, 2, 0, ..., 2, 7, 2]), array([7, 7, 7, ..., 7, 2, 2]), array([5, 7, 0, ..., 7, 2, 2]), array([5, 7, 0, ..., 2, 2, 2]), array([5, 7, 0, ..., 7, 7, 2]), array([7, 2, 0, ..., 2, 2, 2]), array([5, 2, 7, ..., 2, 2, 2]), array([7, 7, 0, ..., 2, 2, 7]), array([5, 2, 7, ..., 2, 2, 7]), array([7, 2, 0, ..., 2, 2, 7])]


In [ ]:
from pandas.core.arrays.arrow import array
from collections import Counter
def create_feature_vectors(arr, max_cls):
  return [[] for _ in range(n)]
  Node_class= range(max_cls)
  counts_out = [np.array([0 for cls in Node_class]) for _ in range(n)]
  counts_in = [np.array([0 for cls in Node_class]) for _ in range(n)]
  counts_2_out = [np.array([0 for cls in Node_class]) for _ in range(n)]
  counts_2_in = [np.array([ 0 for cls in Node_class]) for _ in range(n)]
  list_out = [set() for _ in range(n)]
  list_in = [set() for _ in range(n)]
  for i in range(n):
    for j in range(n):
      # print(j)
      if arr[i][j]==1 and i!=j:
        counts_out[i][arr[j][j]] += 1
        list_out[i].add(j)
      if arr[j][i]==1 and i!=j:
        counts_in[i][arr[j][j]] += 1
        list_in[i].add(j)
  # print(list_in)
  print(Counter([arr[i][i] for i in list_out[0]]))
  # print(counts_in)
  for i in range(n):
    for j in list_out[i]:
      counts_2_out[i] = np.add(counts_2_out[i], counts_out[j])
    for j in list_in[i]:
      counts_2_in[i] = np.add(counts_2_in[i], counts_in[j])
    
  F_vec = [[val for pair in zip(counts_out[i], counts_in[i])
                for val in pair] + [val for pair in zip(counts_2_out[i], 
                                      counts_2_in[i]) for val in pair]
           for i in range(n)]
  return F_vec

import csv
def get_feature_vectors(feat_path):
  with open(feat_path) as f:
    F_vec = list(csv.reader(open(feat_path)))
    F_vec = [[int(j) for j in F_vec[i][1:-2]] for i in range(1, len(F_vec))]
    
    return F_vec

In [ ]:
class_60

[array([1, 1, 3, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([3, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2]),
 array([1, 1, 1, ..., 1, 1, 2])]

In [ ]:
adj_mats = []
adj = A.copy()
# if final_test:
#   for j in range(5):
#     for i in range(n):
#         adj[i][i]=class_60[j][i]
#     adj_mats.append(adj.copy())

if not final_test:
  for i in range(n):
    adj[i][i]=class_60[i]

In [ ]:
print(adj)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaling = StandardScaler()

Scaled_data = data_s.drop('class', axis=1)

scaling.fit(Scaled_data)
Scaled_data = scaling.transform(Scaled_data)

m = 50
principal = PCA(n_components=m)
principal.fit(Scaled_data)
x = principal.transform(Scaled_data)

print(x.shape)

(2708, 50)


In [ ]:
Scaled_data = x

In [ ]:
def train_model(F_vec, p, q, earlystop1, earlystop2, lr):
  # x = np.array(F_vec)
  # k = len(F_vec[0])
  # data=pd.DataFrame({"a_0":x[:,0]})
  # data.insert(loc=1, column="b_0", value=x[:,1])
  # for i in range(1,k-1,2):
  #     data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
  #     data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])
  # print(k)
  # data.insert(loc=k,column='Class',value=data_s['class'].to_numpy())
  # print(data.head(30))

  x = np.array(F_vec)
  k = int((max(catagories)+1)*4)
  data = pd.DataFrame(x)
  # data=pd.DataFrame({"a_0":x[:,0]})
  # data.insert(loc=1, column="b_0", value=x[:,1])
  # for i in range(1,k-1,2):
  #     # print(i)
  #     data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
  #     data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])
  # for i in range(len(F_vec[0])):
  #   data.insert(loc=k+i, column=f'S_{i}', value=x[:,k+i])
  data.insert(loc=len(F_vec[0]),column='Class',value=data_s['class'].to_numpy())

  # feature=["a_0","b_0"]
  # for i in range(1,k-1,2):
  #     feature.append("a_{}".format(int((i+1)/2)))
  #     feature.append("b_{}".format(int((i+1)/2)))

  X=data.drop('Class', axis=1) # Features
  y=data['Class']  # Labels
  print(p)
  X_train=X.iloc[q]
  X_test=X.iloc[p]
  y_train=y.iloc[q]
  y_test=y.iloc[p]

  checkpoint_filepath='./tmp/checkpoint'
  # for train, test in kfold.split(X_train, y_train):
  model = tf.keras.Sequential([
      tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
      tf.keras.layers.Dense(90),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(90),
      tf.keras.layers.Dropout(.2),
      tf.keras.layers.Dense(max(y_train)+1, activation='softmax')
  ])

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=loss_fn, 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  print('------------------------------------------------------------------------')
  history = model.fit(np.array(X_train), np.array(y_train),
                      batch_size = 16,
                      epochs=100,
                      use_multiprocessing=True,
                      workers=4,
                      validation_data=(np.array(X_test), np.array(y_test)),
                      callbacks=[earlystop1, earlystop2,
                                  tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                  save_best_only=True,
                                                                  save_weights_only=True,
                                                                  monitor='val_sparse_categorical_accuracy',
                                                                  mode='max')])
  model.load_weights(checkpoint_filepath)
  scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
  y_pred = [np.argmax(i) for i in model.predict(X_test)]

  return model, scores, y_pred

In [ ]:
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[f'w_{i-1}']) for i in range(1, len(df.columns))] for df in data_by_class.values()]\
        if node_data_name != "data/Pubmed-Diabetes.NODE.paper.tab" else []
sel_basis = [[int(list(df[f'w_{i-1}'].to_numpy()).count(1) >= int(len(df[f'w_{i-1}'].index)*0.1)) 
        for i in range(1, len(df.columns))]
        for df in data_by_class.values()] if node_data_name != "data/Pubmed-Diabetes.NODE.paper.tab" else\
        [[int(np.count_nonzero(list(df.iloc[:, i].astype('float'))) >= int(len(df.index)*0.25)) 
        for i in range(1, len(df.columns))] for df in data_by_class.values()]

data_array = data_s.drop(['ID', 'class'], axis=1).astype(float).to_numpy()
ID_ind = data_s.columns.get_loc('ID')

avg_acc_per_fold = [[],[]]
if final_test:
  earlystop1 = EarlyStopping(patience=7)
  earlystop2 = ReduceLROnPlateau(monitor='val_loss',
                                min_lr = 3e-7, 
                                patience = 4,
                                factor=0.3,
                                verbose = 1)
  mat = np.array(adj)
  for split_ind in range(10):
    np.fill_diagonal(mat, class_60[split_ind])
    acc_per_fold = [[], []]
    loss_per_fold = [[], []]
    for train_iter in range(5):
      
      # F_vec = create_feature_vectors(mat, max(catagories) + 2)

      # # if node_data_name == "data/Pubmed-Diabetes.NODE.paper.tab":
      # #    print("appending PCA feature vector data")
      # #    F_vec = np.hstack((F_vec, Scaled_data))
      # """ TRAINING 1st ITER """
      # model, scores, y_pred = train_model(F_vec, test_splits[split_ind], train_splits[split_ind], earlystop1, earlystop2, 3e-5)
      # print(f'Score for fold {split_ind}, test #{train_iter}, 1st iteration: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
      # acc_per_fold[0].append(scores[1] * 100)
      # loss_per_fold[0].append(scores[0])

      # updated_class_60=np.array(catagories)
      # i=0
      # for d in test_splits[split_ind]:
      #     class_60[split_ind][d]=y_pred[i]
      #     i=i+1
      # print(class_60[split_ind])
      # np.fill_diagonal(mat, class_60[split_ind])
      # print(mat)

      F_vec = create_feature_vectors(mat, max(catagories) + 1)

      # Adding keyword features to F_vec
      if node_data_name == "data/Pubmed-Diabetes.NODE.paper.tab":
         print("appending PCA feature vector data")
         F_vec = np.hstack((F_vec, Scaled_data))
      else:
        print("appending basis + selective feature vector data")
        for i, row in enumerate(data_array):
          index = node_ID[data_s.iloc[i]['ID']]
          f = [int(i > 0) for i in row]
          features = ([np.dot(f, basis[cls]) for cls in range(max(catagories)+1)]\
                    + [np.dot(f,sel_basis[cls]) for cls in range(max(catagories)+1)])\
                    if node_data_name != "data/Pubmed-Diabetes.NODE.paper.tab" else\
                    [np.dot(f,sel_basis[cls]) for cls in range(max(catagories)+1)]
          F_vec[index] += features

      model, scores, y_pred = train_model(F_vec, test_splits[split_ind], train_splits[split_ind], earlystop1, earlystop2, 5e-5)
      print(f'Score for fold {split_ind}, test #{train_iter}, 2nd iteration: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
      acc_per_fold[1].append(scores[1] * 100)
      loss_per_fold[1].append(scores[0])
    # avg_acc_per_fold[0].append(sum(acc_per_fold[0]) / len(acc_per_fold[0]))
    avg_acc_per_fold[1].append(sum(acc_per_fold[1]) / len(acc_per_fold[1]))

else:
  F_vec = create_feature_vectors(adj, max(catagories) + 2) \
              if not os.path.exists("/content/Feature_cora.csv") else get_feature_vectors("/content/Feature_cora.csv")
  x = np.array(F_vec)
  k = len(F_vec[0])
  data=pd.DataFrame({"a_0":x[:,0]})
  data.insert(loc=1, column="b_0", value=x[:,1])
  for i in range(1,k-1,2):
      data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
      data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])
  print(k)
  data.insert(loc=k,column='Class',value=data_s['class'].to_numpy())
  print(data.head(30))
  feature=["a_0","b_0"]
  for i in range(1,k-1,2):
      feature.append("a_{}".format(int((i+1)/2)))
      feature.append("b_{}".format(int((i+1)/2)))

  X=data[feature] # Features
  y=data['Class']  # Labels
  X_train=X.iloc[train_splits[0]]
  X_test=X.iloc[test_splits[0]]
  y_train=y.iloc[train_splits[0]]
  y_test=y.iloc[test_splits[0]]
      

Streaming output truncated to the last 5000 lines.
Epoch 64/100
107/107 [==============================] - 0s 3ms/step - loss: 0.7197 - sparse_categorical_accuracy: 0.7342 - val_loss: 0.5617 - val_sparse_categorical_accuracy: 0.8180 - lr: 5.0000e-05
Epoch 65/100
107/107 [==============================] - 0s 3ms/step - loss: 0.7423 - sparse_categorical_accuracy: 0.7301 - val_loss: 0.5582 - val_sparse_categorical_accuracy: 0.8180 - lr: 5.0000e-05
Epoch 66/100
107/107 [==============================] - 0s 3ms/step - loss: 0.7454 - sparse_categorical_accuracy: 0.7231 - val_loss: 0.5597 - val_sparse_categorical_accuracy: 0.8160 - lr: 5.0000e-05
Epoch 67/100
107/107 [==============================] - 0s 3ms/step - loss: 0.7197 - sparse_categorical_accuracy: 0.7406 - val_loss: 0.5516 - val_sparse_categorical_accuracy: 0.8190 - lr: 5.0000e-05
Epoch 68/100
107/107 [==============================] - 0s 3ms/step - loss: 0.6887 - sparse_categorical_accuracy: 0.7541 - val_loss: 0.5490 - val_sparse_

In [ ]:
[sum(avg_acc_per_fold[i]) / len(avg_acc_per_fold[i]) for i in range(2)]

ZeroDivisionError: ignored

In [ ]:
sum(avg_acc_per_fold[1]) / len(avg_acc_per_fold[1])

83.90000057220459

In [ ]:
max(avg_acc_per_fold)

91.79999828338623

In [ ]:
import statistics
statistics.stdev(avg_acc_per_fold[0])

0.9284660588142666

In [ ]:
statistics.stdev(avg_acc_per_fold[1])

0.6712516005771747

#1st iteration

## XGBoost

In [ ]:
for n_est in range(100, 151, 5):
  bst = XGBClassifier(n_estimators=n_est, max_depth=3, learning_rate=0.3)
  bst.fit(X_train,y_train)
  y_pred=bst.predict(X_test)
  curr_score = metrics.accuracy_score(y_test, y_pred)
  #Import scikit-learn metrics module for accuracy calculation
  
  # Model Accuracy, how often is the classifier correct?
  print("Accuracy for ", n_est, "estimators: ", curr_score*100,"%")

Accuracy for  100 estimators:  84.0 %
Accuracy for  105 estimators:  84.0 %
Accuracy for  110 estimators:  84.0 %
Accuracy for  115 estimators:  83.89999999999999 %
Accuracy for  120 estimators:  83.89999999999999 %
Accuracy for  125 estimators:  83.8 %
Accuracy for  130 estimators:  83.8 %
Accuracy for  135 estimators:  83.8 %
Accuracy for  140 estimators:  83.8 %
Accuracy for  145 estimators:  83.8 %
Accuracy for  150 estimators:  83.8 %


In [ ]:
model = XGBClassifier(n_estimators=115, max_depth=3, learning_rate=0.2)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

## Neural Net Classifier

In [ ]:
kfold = KFold(n_splits=5)

In [ ]:
earlystop1 = EarlyStopping(patience=7)
earlystop2 = ReduceLROnPlateau(monitor='val_loss',
                               min_lr = 3e-7, 
                               patience = 4,
                               factor=0.3,
                               verbose = 1)

In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
checkpoint_filepath='/tmp/checkpoint'
# for train, test in kfold.split(X_train, y_train):
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
    tf.keras.layers.Dense(90),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(90),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(max(y_train)+1, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(3e-5), loss=loss_fn, 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')
history = model.fit(np.array(X_train), np.array(y_train),
                    batch_size = 64,
                    epochs=100,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=(np.array(X_test), np.array(y_test)),
                    callbacks=[earlystop1, earlystop2,
                                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                save_best_only=True,
                                                                save_weights_only=True,
                                                                monitor='val_sparse_categorical_accuracy',
                                                                mode='max')])
model.load_weights(checkpoint_filepath)
scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
10851/10851 [==============================] - 40s 3ms/step - loss: 4.1202 - sparse_categorical_accuracy: 0.2205 - val_loss: 3.0987 - val_sparse_categorical_accuracy: 0.3693 - lr: 3.0000e-05
Epoch 2/100
10851/10851 [==============================] - 33s 3ms/step - loss: 3.1798 - sparse_categorical_accuracy: 0.3271 - val_loss: 2.7891 - val_sparse_categorical_accuracy: 0.3968 - lr: 3.0000e-05
Epoch 3/100
10851/10851 [==============================] - 33s 3ms/step - loss: 2.9163 - sparse_categorical_accuracy: 0.3580 - val_loss: 2.6832 - val_sparse_categorical_accuracy: 0.4041 - lr: 3.0000e-05
Epoch 4/100
10851/10851 [==============================] - 34s 3ms/step - loss: 2.7858 - sparse_categorical_accuracy: 0.3752 - val_loss: 2.6328 - val_sparse_categorical_accuracy: 0.4070 - lr: 3.0000e-05
Epoch 5/100
10851/10851 [==============================] - 34s 3ms/step - loss: 2.7062 - sp

In [ ]:
from sklearn import metrics
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, [np.argmax(i) for i in y_pred])*100,"%")

1311/1311 [==============================] - 2s 1ms/step
Accuracy: 41.796418607978254 %


In [ ]:
max(y_test)

5

In [ ]:
np.argmax(y_pred[1])

4

In [ ]:
y_test

2368    1
895     5
725     2
485     2
634     2
       ..
496     2
497     6
1584    1
1345    6
1627    1
Name: Class, Length: 1083, dtype: int64

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]

In [ ]:
max(y_pred)

6

In [ ]:
adj = A.copy()

In [ ]:
updated_class_60=np.array(catagories)
i=0
for d in test_splits[0]:
    class_60[d]=y_pred[i]
    i=i+1
print(class_60)

[5 2 0 ... 2 2 2]


In [ ]:
for i in range(n):
    adj[i][i]=class_60[i]
print(adj)

[[5 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]]


In [ ]:
max(class_60)

6

In [ ]:
# Node_class=[0,1,2,3,4,5,6]
# F_vec=[]
# for i in range(n):
#     node_F=[]
#     list_out=[]
#     list_In=[]
#     for j in range(n):
#         if A[i][j]==1 and i!=j:
#             list_out.append(A[j][j])
#         if A[j][i]==1 and i!=j:
#             list_In.append(A[j][j])
#     for d in Node_class:
#         count=0
#         count_in=0
#         for k in range(len(list_out)):
#             if list_out[k]==d:
#                 count=count+1
#         node_F.append(count)
#         for k in range(len(list_In)):
#             if list_In[k]==d:
#                 count_in=count_in+1
#         node_F.append(count_in)
#     F_vec.append(node_F)

F_vec = create_feature_vectors(adj, max(catagories) + 1)

Counter()


In [ ]:
adj.shape

(2708, 2708)

# Adding Keyword Features to input vectors

In [ ]:
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}

In [ ]:
data_by_class

In [ ]:
basis = [[max(df[f'w_{i-1}']) for i in range(1, len(df.columns))] for df in data_by_class.values()]

In [ ]:
basis[0][-1]

1

In [ ]:
count0=0
count1=0
for i in range(1433):
    if basis[0][i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

449
984


In [ ]:
sel_basis = [[int(list(df[f'w_{i-1}'].to_numpy()).count(1) >= int(len(df[f'w_{i-1}'].index)*0.1)) 
              for i in range(1, len(df.columns))]
             for df in data_by_class.values()] if node_data_name != "data/Pubmed-Diabetes.NODE.paper.tab" else\
             [[int(np.count_nonzero(list(df.iloc[:, i].astype('float'))) >= int(len(df.index)*0.25)) 
              for i in range(1, len(df.columns))] for df in data_by_class.values()]

In [ ]:
for i in range(1, len(data_by_class[0].columns)):
  print(f'FOR COLUMN {i}:')
  print(data_by_class[0].iloc[:,i])
  print(data_by_class[0].iloc[:,i].astype('float'))

In [ ]:
count0=0
count1=0
for i in range(500):
    if sel_basis[0][i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

467
33


In [ ]:
data_by_class[0]['w_3'].value_counts()

0    295
1      3
Name: w_3, dtype: int64

In [ ]:
count0=0
count1=0
for i in range(1433):
    if sel_basis[0][i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

1414
19


In [ ]:
len(basis)

6

In [ ]:
data_array = data_s.astype(float).to_numpy()  

In [ ]:
ID_ind = data_s.columns.get_loc('ID')

In [ ]:
for row in data_array:
  index = node_ID[str(int(row[ID_ind]))]
  f = [int(i > 0) for i in row[1:-1]]
  features = ([np.dot(f, basis[cls]) for cls in range(max(catagories)+1)]\
             + [np.dot(f,sel_basis[cls]) for cls in range(max(catagories)+1)])\
             if node_data_name != "data/Pubmed-Diabetes.NODE.paper.tab" else\
             [np.dot(f,sel_basis[cls]) for cls in range(max(catagories)+1)]
  F_vec[index] += features

In [ ]:
f

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [ ]:
F_vec

[[0, 0, 0, 2, 0, 0, 0, 0, 0, 8, 0, 0, 8, 8, 9],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 8, 11, 13],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6, 6],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 6, 8],
 [0, 2, 0, 1, 0, 11, 0, 11, 0, 3, 0, 20, 7, 7, 8],
 [2, 9, 1, 2, 2, 2, 8, 12, 6, 2, 0, 6, 12, 13, 16],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 12, 17, 18],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 11, 11, 14],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 6, 7, 7],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 8, 8, 12],
 [0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 9, 10, 12, 14],
 [0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 9, 10, 11],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 6, 7],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 8, 7, 7],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 9],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 7, 0, 0, 7, 9, 10],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 8, 13, 12],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 7, 6, 6],
 [1, 0, 1, 1, 0, 0, 10, 0, 6, 5, 0, 0, 5, 5, 5],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 6, 9, 9],
 [0, 0, 1, 0, 

# 2nd Iteration training

In [ ]:
x

array([[ 0,  0,  0, ...,  8,  8,  9],
       [ 0,  0,  0, ...,  8, 11, 13],
       [ 0,  0,  0, ...,  4,  6,  6],
       ...,
       [ 0,  0,  0, ...,  7,  7,  8],
       [ 0,  0,  0, ..., 13, 15, 20],
       [ 0,  0,  0, ...,  4,  6,  9]])

In [ ]:
x = np.array(F_vec)
k = int((max(catagories)+1)*4)
data=pd.DataFrame({"a_0":x[:,0]})
data.insert(loc=1, column="b_0", value=x[:,1])
for i in range(1,k-1,2):
    # print(i)
    data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
    data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])
for i in range(len(F_vec[0]) - k):
  data.insert(loc=k+i, column=f'S_{i}', value=x[:,k+i])
data.insert(loc=len(F_vec[0]),column='Class',value=data_s['class'].to_numpy())
data.head(30)

,a_0,b_0,a_1,b_1,a_2,b_2,a_3,b_3,a_4,b_4,...,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,Class
0,0,0,0,0,0,0,0,0,0,0,...,24,24,5,3,2,3,3,6,5,5
1,0,0,0,0,3,1,0,0,0,0,...,7,8,1,3,3,3,2,0,1,2
2,1,0,0,0,0,0,0,0,0,0,...,20,18,5,2,1,1,1,3,1,0
3,0,0,2,1,0,0,0,0,0,0,...,7,6,1,2,1,2,1,2,1,1
4,0,0,0,0,1,2,0,0,0,0,...,13,16,3,5,4,3,4,2,2,2
5,2,3,0,0,0,0,0,0,0,0,...,20,20,3,3,1,1,2,3,2,0
6,0,2,0,0,0,0,0,0,0,0,...,4,3,1,1,1,1,1,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,10,20,0,1,0,1,1,1,2,6
8,0,0,0,0,0,0,0,0,0,0,...,10,19,2,3,2,2,3,3,4,6
9,0,0,0,0,0,0,0,0,0,0,...,18,17,2,2,2,1,2,3,2,5


In [ ]:
# feature=["a_0","b_0"]
# for i in range(1,k-1,2):
#     feature.append("a_{}".format(int((i+1)/2)))
#     feature.append("b_{}".format(int((i+1)/2)))

X=data.drop('Class', axis=1) # Features
y=data['Class']  # Labels
X_train=X.iloc[q]
X_test=X.iloc[p]
y_train=y.iloc[q]
y_test=y.iloc[p]

In [ ]:
for n_est in range(100, 151, 5):
  bst = XGBClassifier(n_estimators=n_est, max_depth=3, learning_rate=0.07)
  bst.fit(X_train,y_train)
  y_pred=bst.predict(X_test)
  curr_score = metrics.accuracy_score(y_test, y_pred)
  #Import scikit-learn metrics module for accuracy calculation
  
  # Model Accuracy, how often is the classifier correct?
  print("Accuracy for ", n_est, "estimators: ", curr_score*100,"%")

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)

In [ ]:
data = pd.read_csv('/content/Feature_nbd_mag.csv', index_col=0)

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,689,690,691,692,693,694,695,696,697,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,246
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,189
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95


In [ ]:
for i, c in enumerate(data['Class'].isnull().values):
  if c:
    print(i)

In [ ]:
Node_Class = graph[1]['paper'][:,0]

In [ ]:
data['Class'] = Node_Class

In [ ]:
Node_Fec = list(graph[0]['node_feat_dict']['paper'])
feature_names = ["w_{}".format(ii) for ii in range(128)]
Node_Fec = pd.DataFrame(Node_Fec,columns=feature_names)

In [ ]:
Node_Fec.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_118,w_119,w_120,w_121,w_122,w_123,w_124,w_125,w_126,w_127
0,-0.095379,0.040758,-0.210948,-0.064362,-0.225940,-0.002323,-0.555826,-0.352909,0.040634,-0.195226,...,0.146386,-0.378653,-0.328871,-0.136878,-0.007779,-0.157634,-0.069693,0.061569,-0.027663,-0.133832
1,-0.151047,-0.107315,-0.221964,-0.034725,0.010814,-0.009673,-0.434323,-0.461733,0.112732,0.091751,...,0.047749,-0.065133,-0.208891,-0.366014,-0.045869,-0.297474,0.056755,0.345754,-0.027737,-0.218527
2,-0.114799,-0.175982,-0.260556,0.011920,-0.129273,-0.044518,-0.345423,-0.378261,-0.088449,-0.091060,...,0.047763,-0.394512,-0.238776,0.080928,0.057844,-0.205098,-0.067513,0.173058,-0.156445,-0.277954
3,0.004506,0.042368,-0.178465,-0.138479,-0.138268,0.024902,-0.438153,-0.450127,0.066984,-0.017753,...,0.116500,-0.255289,-0.278228,-0.081235,0.251969,-0.066206,0.095656,0.282228,0.016337,-0.212731
4,-0.094474,-0.080044,-0.222468,-0.158952,-0.024708,0.004351,-0.463666,-0.344648,-0.101808,-0.012426,...,-0.117917,-0.139355,-0.228969,-0.062332,0.132850,-0.269122,0.024864,0.288779,-0.186985,-0.205087


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaling = StandardScaler()

Scaled_data = Node_Fec

scaling.fit(Scaled_data)
Scaled_data = scaling.transform(Scaled_data)

m = 50
principal = PCA(n_components=m)
principal.fit(Scaled_data)
x = principal.transform(Scaled_data)

print(x.shape)

(736389, 50)


In [ ]:
Scaled_data = pd.DataFrame(x)

In [ ]:
Scaled_data.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-3.082088,-3.770587,0.435245,5.163740,0.475839,0.605648,-0.021783,2.444352,1.843078,0.596841,...,0.079877,-0.421684,0.577886,0.675871,0.317556,0.489458,0.695124,0.216502,0.288270,0.049476
1,0.728420,-2.530880,-2.933542,0.058950,3.020178,2.115058,0.902393,0.766614,1.266361,0.704747,...,0.662028,-0.053322,1.400128,0.167979,-0.548252,-0.075149,-0.206946,-0.419678,-0.878682,0.092751
2,-5.002440,5.485702,-1.026681,0.128345,-0.158239,-0.279623,2.143083,2.579699,1.360662,0.643789,...,0.089046,0.193530,1.257107,-0.758059,0.343196,0.729315,0.628578,0.103256,1.271947,0.382716
3,-1.776126,-3.535830,0.390636,0.905836,0.544934,2.867392,1.611869,-0.885575,1.645747,1.556516,...,-0.691473,-1.383707,0.888929,0.680112,-0.544849,1.186543,0.049447,0.043830,-0.440982,-1.941840
4,-1.486382,-1.171743,-1.731701,-0.380522,-1.250264,0.951093,2.504003,1.279308,0.567930,1.005565,...,0.361851,-0.307620,0.215941,0.266164,-0.103523,0.545216,-0.205958,0.311127,-0.758992,-0.590508


In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,689,690,691,692,693,694,695,696,697,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,246
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,189
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,131
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,95


In [ ]:
Data = pd.concat([data, Node_Fec], axis=1)

In [ ]:
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,w_118,w_119,w_120,w_121,w_122,w_123,w_124,w_125,w_126,w_127
0,0,0,0,0,0,0,0,0,0,0,...,0.146386,-0.378653,-0.328871,-0.136878,-0.007779,-0.157634,-0.069693,0.061569,-0.027663,-0.133832
1,0,0,0,0,0,0,0,0,0,0,...,0.047749,-0.065133,-0.208891,-0.366014,-0.045869,-0.297474,0.056755,0.345754,-0.027737,-0.218527
2,0,0,0,0,0,0,0,0,0,0,...,0.047763,-0.394512,-0.238776,0.080928,0.057844,-0.205098,-0.067513,0.173058,-0.156445,-0.277954
3,0,0,0,0,0,0,0,0,0,0,...,0.116500,-0.255289,-0.278228,-0.081235,0.251969,-0.066206,0.095656,0.282228,0.016337,-0.212731
4,0,0,0,0,0,0,0,0,0,0,...,-0.117917,-0.139355,-0.228969,-0.062332,0.132850,-0.269122,0.024864,0.288779,-0.186985,-0.205087


In [ ]:
Node_Year=graph[0]['node_year']['paper']
Node_Year_list=list(Node_Year[:,0])

E=graph[0]['edge_index_dict'][('paper', 'cites', 'paper')]
EdgeL=E[0]
EdgeR=E[1]
print(EdgeL)
print(EdgeR)
print(np.unique(Node_Year_list))

Node_Year_list.index(2019)
list_size = len(Node_Year_list)
NIndex_19=[]
# declare for loop
for itr in range(list_size):
 
      # check the condition
    if(Node_Year_list[itr] == 2019):
        NIndex_19.append(itr)
 
          # print the indices
print(len(NIndex_19))

[     0      0      0 ... 736388 736388 736388]
[    88  27449 121051 ... 421711 427339 439864]
[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
41939


In [ ]:
N_class_train=Node_Class
for d in NIndex_19:
    N_class_train[d]=500
print(N_class_train)

[246 131 189 ... 266 289   1]


In [ ]:
N_class_train=Node_Class
y_pred = [np.argmax(i) for i in y_pred]
for i, d in enumerate(NIndex_19):
  N_class_train[d] = y_pred[i]

In [ ]:
len(N_class_train)

736389

In [ ]:
F_vec = n*classes
# for i in range(len(EdgeL)):
  left, right = EdgeL[i], EdgeR[i]
  get_class(right)
  F_vec[left][get_class(right)] += 1

  #O(E)
  

In [ ]:
def CountX(lst,x):
    return(lst.count(x))

Node_class=list(range(349))
F_vec=[]
n=736389
for i in range(n):
    print("\rProcessing file {} ({}%)".format(i, 100*i//n), end='', flush=True)
    node_F=[]
    list_out=[]
    list_In=[]
    #S_nbd_out=[]
    #S_nbd_in=[]
    indx=np.where(EdgeL==i)
    indxIn=np.where(EdgeR==i)
    #print(indx)
    for j in indx[0]:
        list_out.append(N_class_train[EdgeR[j]])
   # indx[0].clear()
    for k in indxIn[0]:
        list_In.append(N_class_train[EdgeL[k]])
   # indxIn[0].clear()

                
                
    for d in Node_class:
        node_F.append(CountX(list_out,d))
        node_F.append(CountX(list_In,d))
    F_vec.append(node_F)


Processing file 736388 (99%)

In [ ]:
data = pd.DataFrame(F_vec)
data.insert(loc=698, column='Class', value=Node_Class)


In [ ]:
Data = pd.concat([data, Node_Fec], axis=1)

In [ ]:
data.columns.get_loc('Class')

698

In [ ]:
len(Node_Class)

736389

In [ ]:
with open('/content/Feature_nbd_mag.csv') as f:
  lines = f.readlines()
  line = lines[96970].split(',')
  print(len(lines))

96971


In [ ]:
X = Data.drop('Class', axis=1)
y = Data['Class']
X_train=X.iloc[list(train_idx['paper'])+list(valid_idx['paper'])]
X_test=X.iloc[test_idx['paper']]
y_train=y.iloc[list(train_idx['paper'])+list(valid_idx['paper'])]
y_test=y.iloc[test_idx['paper']]

In [ ]:
earlystop1 = EarlyStopping(patience=7)
earlystop2 = ReduceLROnPlateau(monitor='val_loss',
                               min_lr = 3e-7, 
                               patience = 4,
                               factor=0.3,
                               verbose = 1)

In [ ]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,w_118,w_119,w_120,w_121,w_122,w_123,w_124,w_125,w_126,w_127
359,0,0,0,0,0,0,0,0,0,0,...,0.098125,-0.268199,-0.205214,-0.097784,0.193881,-0.225376,-0.054646,0.123194,0.025784,-0.188598
411,0,0,6,0,0,0,0,0,0,0,...,-0.070180,-0.256800,-0.201145,-0.040072,0.137642,-0.249818,0.019517,0.283035,-0.211814,-0.444755
608,0,0,30,0,0,0,0,0,0,0,...,0.227743,-0.848724,-0.380617,0.224889,-0.349129,-0.437328,0.015382,0.271330,-0.378993,-0.388307
873,0,0,0,0,0,0,0,0,0,0,...,0.015031,-0.349369,-0.389953,0.037686,0.309974,-0.082996,0.041261,0.167244,-0.091509,-0.015484
11951,0,0,2,0,0,0,0,0,0,0,...,-0.026104,-0.084677,-0.135089,-0.213256,0.223176,-0.161862,0.032179,0.279161,-0.085118,-0.231291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736349,0,0,0,0,0,0,0,0,0,0,...,-0.090542,-0.163129,-0.285086,-0.358813,0.259896,0.033160,0.008394,0.146009,0.122744,-0.156794
736354,0,0,0,0,0,0,0,0,0,0,...,-0.115263,-0.148100,-0.342683,0.172944,0.077405,0.099258,0.026068,0.458861,-0.165574,-0.159797
736358,0,0,0,0,0,0,0,0,0,0,...,-0.023282,-0.202687,-0.163427,0.055721,0.159572,0.089135,0.016526,0.169853,-0.309299,-0.111012
736384,0,0,0,0,0,0,0,0,0,0,...,0.043255,-0.065534,-0.255169,-0.117872,0.276171,-0.185582,-0.005036,0.197039,-0.060304,-0.160619


In [ ]:
min(Node_Class)

0

In [ ]:
y_test

359       500
411       500
608       500
873       500
11951     500
         ... 
736349    500
736354    500
736358    500
736384    500
736385    500
Name: Class, Length: 41939, dtype: int64

In [ ]:
acc_per_fold = []
loss_per_fold = []
fold_no = 1
checkpoint_filepath='/tmp/checkpoint'
# for train, test in kfold.split(X_train, y_train):
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(X_train.iloc[0]),)),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(max(y_train)+1, activation='softmax')
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(6e-5), loss=loss_fn, 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
print('------------------------------------------------------------------------')
print(f'Training for fold {fold_no} ...')
history = model.fit(np.array(X_train), np.array(y_train),
                    batch_size = 64,
                    epochs=100,
                    use_multiprocessing=True,
                    workers=4,
                    validation_data=(np.array(X_test), np.array(y_test)),
                    callbacks=[earlystop1, earlystop2,
                                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                save_best_only=True,
                                                                save_weights_only=True,
                                                                monitor='val_sparse_categorical_accuracy',
                                                                mode='max')])
model.load_weights(checkpoint_filepath)
scores = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])
fold_no += 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/100
10851/10851 [==============================] - 34s 3ms/step - loss: 3.5877 - sparse_categorical_accuracy: 0.2834 - val_loss: 2.7656 - val_sparse_categorical_accuracy: 0.3953 - lr: 6.0000e-05
Epoch 2/100
10851/10851 [==============================] - 33s 3ms/step - loss: 2.8138 - sparse_categorical_accuracy: 0.3721 - val_loss: 2.6073 - val_sparse_categorical_accuracy: 0.4094 - lr: 6.0000e-05
Epoch 3/100
10851/10851 [==============================] - 33s 3ms/step - loss: 2.6519 - sparse_categorical_accuracy: 0.3939 - val_loss: 2.5581 - val_sparse_categorical_accuracy: 0.4105 - lr: 6.0000e-05
Epoch 4/100
10851/10851 [==============================] - 33s 3ms/step - loss: 2.5752 - sparse_categorical_accuracy: 0.4035 - val_loss: 2.5259 - val_sparse_categorical_accuracy: 0.4126 - lr: 6.0000e-05
Epoch 5/100
10851/10851 [==============================] - 33s 3ms/step - loss: 2.5305 - sp